# Manipulating models

In [3]:
from cameo import load_model
model = load_model('data/e_coli_core.xml.gz')

## Traveling in time

Usually one relies on making copies if objects need to be changed but the original state needs to be retained. Unfortunately, making copies of models is time consuming.

In [4]:
model.solve()

,fluxes,reduced_costs
ACALD,0.000000,6.938894e-18
ACALDt,0.000000,0.000000e+00
ACKr,0.000000,1.040834e-17
ACONTa,6.007250,0.000000e+00
...,...,...
THD2,0.000000,-2.546243e-03
TKT1,1.496984,-1.387779e-17
TKT2,1.181498,1.387779e-17
TPI,7.477382,-6.938894e-18


In [5]:
%%time
copy_of_model = model.copy()

CPU times: user 21.9 ms, sys: 4.02 ms, total: 25.9 ms
Wall time: 37.3 ms


Yes, even milliseconds add up pretty quickly if you need to run many simulation (e.g. if you need to knock out every single gene individually in the model to check if it is essential or not).

In [6]:
%%time
for gene in model.genes:
    mutant = model.copy()
    mutant.genes.get_by_id(gene.id).knock_out()

CPU times: user 2.96 s, sys: 180 ms, total: 3.14 s
Wall time: 4.27 s


For that reason cameo provides a mechanism that is less time consuming. Some methods (e.g. `knock_out`) provide a keyword argument that allows the user to provide a `TimeMachine` object (`tm`). TimeMachine can be used in combination with Python's `with` statement in the same way you would use it open a file for example.

In [13]:
from cameo.util import TimeMachine

In [14]:
%%time
with TimeMachine() as tm:
    for gene in model.genes:
        gene.knock_out(time_machine=tm)

CPU times: user 39.3 ms, sys: 1.68 ms, total: 41 ms
Wall time: 41.7 ms


This is very similar to opening, reading and closing of files in Python.

In [18]:
with open('data/ecoli.csv', 'r') as file:
    print(file.read()[0:103])

bnumber,genes
b0001,thrL
b0002,thrA
b0002,thrA1
b0002,thrA2
b0003,thrB
b0004,thrC
b0005,yaaX
b0006,yaaA


Once one leaves the scope of `with` (the lines start again one indentation level to the right), everything is rolled back (the open file is closed; the gene knockout is undone).

## Changing flux bounds

One can change the flux bounds of a reaction using the 'change_bounds' methods. For example let's set the lower and upper bound of phosphoglycerate kinase to 0 (effictively knocking out the reaction).

In [19]:
with TimeMachine() as tm:
    model.reactions.PGK.change_bounds(lb=0, ub=0, time_machine=tm)
    print("PGK's bounds inside the with statement")
    print(model.reactions.PGK.lower_bound, model.reactions.PGK.upper_bound)
    print('Mutant growth rate: ', model.solve().objective_value)
print("PGK's bounds outside the with statement")
print(model.reactions.PGK.lower_bound, model.reactions.PGK.upper_bound)

PGK's bounds inside the with statement
0 0
Mutant growth rate:  -2.171174082614637e-15
PGK's bounds outside the with statement
-1000.0 1000.0


## Changing the medium

One can access the medium condition using `model.medium`. Lower and upper bounds of exchanges reactions are used to set a certain medium condition.

In [3]:
model.medium

,reaction_id,reaction_name,lower_bound,upper_bound
0,EX_co2_e,CO2 exchange,-1000.0,1000.0
1,EX_glc__D_e,D-Glucose exchange,-10.0,1000.0
2,EX_h_e,H+ exchange,-1000.0,1000.0
3,EX_h2o_e,H2O exchange,-1000.0,1000.0
4,EX_nh4_e,Ammonia exchange,-1000.0,1000.0
5,EX_o2_e,O2 exchange,-1000.0,1000.0
6,EX_pi_e,Phosphate exchange,-1000.0,1000.0


In [4]:
from cameo.util import TimeMachine

In [20]:
with TimeMachine() as tm:
    model.reactions.EX_glc__D_e.change_bounds(lb=0, ub=0, time_machine=tm)
    model.reactions.EX_succ_e.change_bounds(lb=-10, ub=0, time_machine=tm)
    solution = model.solve()
    print(model.reactions.BIOMASS_Ecoli_core_w_GAM.flux)

0.3975630154277604


## Exercise

* Change the carbon source in the medium to a different carbon source. What is the difference in the growth rate observed?

## Adding reactions and pathways

In [5]:
from cameo import Reaction

In [10]:
reaction = Reaction('Blah')
model.add_reaction(reaction)
reaction.reaction = 'g3p_c <=> gold_c'

unknown metabolite 'gold_c' created


In [14]:
model.reaction_for??

In [13]:
model.metabolites.gold_c.compartment

## Exercise

* Add a pathway to the model